# AI COURSE - FIRST ASSIGNMENT
## MUTLICLASS CLASSIFICATION PROBLEM WITH IMAGES 
We re-do the steps from the previous notebook for augmented images and re-train the models

### == IMPORTS ==========================================


**general imports**

In [ ]:
import os 
import shutil 
import random 
import pathlib

import numpy as np
import matplotlib.pyplot as plt 
import cv2 
import PIL

**tensorflow imports**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, losses, datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, \
    Activation, Flatten, Dense, Lambda, MaxPooling2D, Dropout

### == CHECKING THE DATA ================================


**making the path**

In [ ]:
data_dir = '.\\data'
data_dir = pathlib.Path(data_dir)
data_dir

**listing the images**

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]

**how many images do we have**

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

**loading the image paths**

In [ ]:
bikes = list(data_dir.glob('bike/*'))
buses = list(data_dir.glob('bus/*')) 
cars  = list(data_dir.glob('car/*'))
print(bikes[5])
print(buses[5])
print(cars[5])

**looking at samples**

In [ ]:
PIL.Image.open(str(bikes[1]))

In [ ]:
PIL.Image.open(str(buses[8]))

In [ ]:
PIL.Image.open(str(cars[1]))

### == LOADING THE DATA ================================


**constructing image path dictionary**

In [ ]:
vehicles_images_dict = {
    'bikes': list(data_dir.glob('bike/*')),
    'buses': list(data_dir.glob('bus/*')),
    'cars':  list(data_dir.glob('car/*')),
}

**constructing label dictionary**

In [ ]:
vehicles_labels_dict = {
    'bikes': 0, 
    'buses': 1,
    'cars': 2,
}

**double checking the dictionary**

In [ ]:
vehicles_images_dict['bikes'][:5]

In [ ]:
str(vehicles_images_dict['bikes'][0])

**loading the images into numpy arrays**

In [ ]:
X, y = [], []

for vehicle_name, images in vehicles_images_dict.items():
    for image in images: 
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (32, 32))
        X.append(resized_img)
        y.append(vehicles_labels_dict[vehicle_name])

X = np.array(X)
y = np.array(y)

In [ ]:
y.shape

### == SPLITTING DATA ====================================


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### == LENET-5 MODEL ====================================
#### (WITHOUT IMAGE AUGMENTATION)


**initializing the model and adding the layers**

In [ ]:
num_classes = 3

model = Sequential()

model.add(Conv2D(6, 5, activation='tanh', input_shape=X_train.shape[1:]))
model.add(AveragePooling2D(2))
model.add(Activation('sigmoid'))
model.add(Conv2D(16, 5, activation='tanh'))
model.add(AveragePooling2D(2))
model.add(Activation('sigmoid'))
model.add(Conv2D(120, 5, activation='tanh'))
model.add(Flatten())
model.add(Dense(84, activation='tanh'))
model.add(Dense(num_classes, activation='softmax'))

# checking the model layers
model.summary()

**compiling the model**

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

**fitting the model**

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))

**plotting the loss function and accuracy over epochs**

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

**evaluation of the model**

In [ ]:
model.evaluate(X_test, y_test)

### == ALEXNET MODEL ===================================
#### (WITHOUT IMAGE AUGMENTATION)

**initializing the model and adding the layers**

In [ ]:
model = models.Sequential()

model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=X_train.shape[1:]))
model.add(Conv2D(96, 11, strides=4, padding='same'))
model.add(Lambda(tf.nn.local_response_normalization))
model.add(Activation('relu'))
model.add(MaxPooling2D(3, strides=2))
model.add(Conv2D(256, 5, strides=4, padding='same'))
model.add(Lambda(tf.nn.local_response_normalization))
model.add(Activation('relu'))
model.add(MaxPooling2D(3, strides=2))
model.add(Conv2D(384, 3, strides=4, padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(384, 3, strides=4, padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, 3, strides=4, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.summary()

**compiling the model**

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

**fitting the model**

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))

**plotting the loss function and accuracy over epochs**

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
model.evaluate(X_test, y_test)

### == IMAGE AUGMENTATION ==============================



**defining a data augmentation function**

In [ ]:
img_height = 32
img_width = 32
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

**checking the difference of original image and augmented image**

In [ ]:
plt.axis('off')
plt.imshow(X[5])

In [ ]:
plt.axis('off')
plt.imshow(data_augmentation(X)[5].numpy().astype("uint8"))

### == LENET-5 MODEL ====================================
#### (USING IMAGE AUGMENTATION)


In [ ]:
num_classes = 3

model = Sequential()

model.add(data_augmentation)
model.add(Conv2D(6, 5, activation='tanh', input_shape=X_train.shape[1:]))
model.add(AveragePooling2D(2))
model.add(Activation('sigmoid'))
model.add(Conv2D(16, 5, activation='tanh'))
model.add(AveragePooling2D(2))
model.add(Activation('sigmoid'))
model.add(Conv2D(120, 5, activation='tanh'))
model.add(Flatten())
model.add(Dense(84, activation='tanh'))
model.add(Dense(num_classes, activation='softmax'))

# checking the model layers
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

In [ ]:
model.evaluate(X_test, y_test)